# tensorcircuit SDK for QCloud（230109 ver）

## import the package

``apis`` is temporarily as the entry point submodule for qcloud

In [ ]:
import tensorcircuit as tc
from tensorcircuit.cloud import apis
import numpy as np

## setup the token

The users need an API token from tQuK to connect to the server and submit tasks, the token only need be set once and it is then written to the local computer

In [ ]:
apis.set_token("foobar")
# only required running once for a given laptop

## list providers/devices/properties

In [ ]:
apis.list_providers()

In [ ]:
apis.list_devices("tencent")

In [ ]:
apis.list_properties(device="9gmon")

In [ ]:
apis.list_properties(device="simulator:tcn1")

## Task submit and the results

In [ ]:
c = tc.Circuit(1)
c.H(0)

t = apis.submit_task(device="simulator:tc", circuit=c, shots=1024)
print(t.details())
t.results(blocked=True)

``blocked=True`` can wait until the task is finished or failed (rasing an error)

In [ ]:
t.status()

In [ ]:
t.get_device()

In [ ]:
# resubmit a job
t1 = t.resubmit()
t1.details(blocked=True, prettify=True)

``t.details`` can also permit the ``blocked=True`` option, which waits until the task is finished or failed (no error raised).

Also note by using ``prettfiy=True`` option, we have python datatime object for the timestamp which is easy to read but hard for io (not directly json serializable anymore) 

## local provider enable quick debugging and testing

In [ ]:
apis.set_provider("local")
# using tc simulator on local device: your own laptop is your server
apis.list_devices()

In [ ]:
c = tc.Circuit(2)
c.h(0)
c.cx(0, 1)

# exactly the same API as tQuK
t = apis.submit_task(circuit=c, device="testing", shots=8192)
t.results(blocked=True)

In [ ]:
tl = apis.list_tasks()
tl

In [ ]:
id_ = tl[0].__str__()
print(id_)
t = apis.get_task(id_)
t.details()

In [ ]:
id_ = tl[0].__str__()
print(id_.split("~~")[1])
t = apis.get_task(id_)
t.details()

The task can indexed either with device information or not (as long as we use ``set_provider``)

In [ ]:
# back to tencent server for demonstration below
apis.set_provider("tencent")

## GHZ on real device and readout mitigation

In [ ]:
nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.H(8)
c.cnot(8, 4)
c.cnot(4, 0)
c.cnot(0, 2)
c.cnot(2, 6)

t = apis.submit_task(
    circuit=c, shots=shots, device="9gmon", enable_qos_qubit_mapping=False
)
raw_count = t.results(blocked=True)
# blocked = True will block the process until the result is returned
# the default behavior is blocked=False, where only one query is made and raise error when the task is incomplete

In the below, we use tensorcircuit builtin powerful tool for readout mitigation: ``tc.results.readout_mitigation.ReadoutMit``, it supports various method for calibriation and mitigation

In [ ]:
ReadoutMit = tc.results.readout_mitigation.ReadoutMit
mit = ReadoutMit("9gmon?o=0")
mit.cals_from_system(nqubit, shots, method="local")
miti_count = mit.apply_correction(raw_count, nqubit, "square")

By attaching ``?o=0`` after the device string, we have the same effect of setting ``enable_qos_qubit_mapping=False`` (o=1)
and ``enable_qos_gate_decomposition=False`` (o=2), and both of them of by default True (o=3)

In [ ]:
raw_count = tc.results.counts.marginal_count(raw_count, [8, 4, 0, 2, 6])
miti_count = tc.results.counts.marginal_count(miti_count, [8, 4, 0, 2, 6])
# only keep the result for qubit 8, 4, 0, 2, 6 and in that exact order

In [ ]:
from qiskit.visualization import plot_histogram

plot_histogram([raw_count, miti_count])

In [ ]:
ideal_count = tc.results.counts.vec2count(c.probability(), prune=True)
# we can obtain analytic count results by ``c.probability()`` method, and ``vec2count`` with transform the vector as a dict

ideal_count = tc.results.counts.marginal_count(ideal_count, [8, 4, 0, 2, 6])
tc.results.counts.kl_divergence(
    ideal_count, raw_count
), tc.results.counts.kl_divergence(ideal_count, miti_count)

In [ ]:
# we can directly check local readout matrix on each qubit
print("readout matrix")
for i, m in enumerate(mit.single_qubit_cals):
    print("qubit %s:" % i)
    print(m)

batch submission is possible with multiple circuits in a list and the return is a list of task, respectively

In [ ]:
# we can also do a batch submission for the real hardware chip, simply by provide a circuit list

c = tc.Circuit(2)
c.h(0)

c1 = tc.Circuit(2)
c1.h(1)

ts = apis.submit_task(device="9gmon", circuit=[c, c1], shots=1024)

for t in ts:
    print(t.results(blocked=True))

## three approaches for measure on partial of the qubits

Note the return order should ideally follow the measure order in the instructions (wait to be fixed both on simulator backend and on the return from the real chips, can skip this section for now)

In [ ]:
# directly partial measure

# approach 1
nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.x(8)
c.x(6)

t = apis.submit_task(circuit=c, shots=shots, device="9gmon?o=0", measure=[8, 2, 6])
print(t.results(blocked=True))

In [ ]:
# directly partial measure

# approach 2
from qiskit.circuit import QuantumCircuit

qc = QuantumCircuit(9, 9)
qc.x(8)
qc.x(6)
qc.measure(8, 8)
qc.measure(2, 2)
qc.measure(6, 6)

t = apis.submit_task(circuit=qc, shots=shots, device="9gmon?o=0")
print(t.results(blocked=True))

 The above case also indicates that tc ``submit_task`` API directly support Qiskit ``QuantumCircuit`` object

In [ ]:
# directly partial measure

# approach 3, recommended approach

nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.x(8)
c.x(6)
c.measure_instruction(8)
c.measure_instruction(2)
c.measure_instruction(6)

t = apis.submit_task(circuit=c, shots=shots, device="9gmon?o=0")
print(t.results(blocked=True))

In [ ]:
# partial measurment also supported via the simulator

In [ ]:
nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.x(8)
c.x(6)
c.measure_instruction(8)
c.measure_instruction(2)
c.measure_instruction(6)

t = apis.submit_task(circuit=c, shots=shots, device="simulator:tc")
print(t.results(blocked=True))

In [ ]:
nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.x(8)
c.x(6)
c.measure_instruction(8)
c.measure_instruction(2)
c.measure_instruction(6)

t = apis.submit_task(circuit=c, shots=shots, device="simulator:aer")
print(t.results(blocked=True))

## two level compiling system

We provide compiling support at frond end (via tc-qiskit pipeline) and at back end (in qos).
The front end option is enabled by ``compiled-True`` (default to False) and also with an optional dict for ``qiskit.transpile`` arguments called ``compiled_options``. The backend qos compiling is controlled by ``enable_qos_qubit_mapping`` and ``enable_qos_gate_decomposition`` (all default to True). The ``?o=int`` str after the device name can overide qos compiling options.

In [ ]:
# directly use built-in mitigation with expectation evaluation + front-end (tc/qiskit) compiling system

nqubit = 3
shots = 8192
c = tc.Circuit(nqubit)
c.h(0)
c.h(1)
c.rx(2, theta=0.7)
c.ry(1, theta=-1.2)
c.cnot(0, 1)
c.cnot(2, 0)
c.h(1)

print("exact: ", [np.real(c.expectation_ps(z=[i])) for i in range(nqubit)])
t = apis.submit_task(
    circuit=c,
    shots=shots,
    device="9gmon",
    compiled=True,
    enable_qos_qubit_mapping=False,
    enable_qos_gate_decomposition=False,
)

ct = t.results(blocked=True)

mit = tc.results.readout_mitigation.ReadoutMit("9gmon?o=0")
mit.cals_from_system(3, method="local")

print(
    "experiments (mitigated): ",
    [mit.expectation(ct, [i]) for i in range(nqubit)],
)

In [ ]:
c.draw()

In [ ]:
# use backend compiling system enabled by qos

nqubit = 3
shots = 8192
c = tc.Circuit(nqubit)
c.h(0)
c.h(1)
c.rx(2, theta=0.7)
c.ry(1, theta=-1.2)
c.cnot(0, 1)
c.cnot(2, 0)
c.h(1)

print("exact: ", [np.real(c.expectation_ps(z=[i])) for i in range(nqubit)])

t = apis.submit_task(
    circuit=c,
    shots=shots,
    device="9gmon",
    compiled=False,
    enable_qos_qubit_mapping=True,
    enable_qos_gate_decomposition=True,
)

ct = t.results(blocked=True)

mit = tc.results.readout_mitigation.ReadoutMit("9gmon")
mit.cals_from_system(3, method="local")

print(
    "experiments (mitigated): ",
    [mit.expectation(ct, [i]) for i in range(nqubit)],
)

In [ ]:
# inspect compiling results from the tc and qos for the task, we can directly get the circuit objects from prettified details

c_complied_before_qos = t.details(prettify=True)["frontend"]
c_complied_after_qos = t.details(prettify=True)["backend"]

In [ ]:
c_complied_before_qos.draw()

In [ ]:
c_complied_after_qos.draw()

dry run mode to query compiled circuit only from qos (not really sending the circuit to chips), we can use ``qos_dry_run=True`` option


In [ ]:
nqubit = 3
shots = 8192
c = tc.Circuit(nqubit)
c.h(0)
c.h(1)
c.rx(2, theta=0.7)
c.ry(1, theta=-1.2)
c.cnot(0, 1)
c.cnot(2, 0)
c.h(1)

print("exact: ", [np.real(c.expectation_ps(z=[i])) for i in range(nqubit)])

t = apis.submit_task(
    circuit=c,
    shots=shots,
    device="9gmon",
    compiled=True,
    enable_qos_qubit_mapping=True,
    enable_qos_gate_decomposition=True,
    qos_dry_run=True,
)

In [ ]:
t.details(prettify=True)["backend"].draw()

## scalable readout simulation and mitigation

Via TensorCircuit, we provide the capability to do scalable (20+ qubits) readout error simulation and mitigation

In [ ]:
# scalable readout error simulation on tQuK with tensorcircuit backend using tensor network approach

c = tc.Circuit(3)
t = apis.submit_task(circuit=c, device="simulator:tcn1", shots=8192)
t.results(blocked=True)

In [ ]:
t.results(mitigated=True)

In [ ]:
c = tc.Circuit(25)
t = apis.submit_task(circuit=c, device="simulator:tcn1", shots=8192)
t.results(blocked=True)

Simulator device also support batch submission

In [ ]:
# batch submission to the simulator
cs = []
for i in range(15):
    c = tc.Circuit(15)
    c.x(i)
    cs.append(c)
ts = apis.submit_task(circuit=cs, device="simulator:tcn1", shots=8192)

In [ ]:
# mitigated with m3 scalable on count dict
c = tc.Circuit(15)
c.x(0)
t = apis.submit_task(circuit=c, device="simulator:tcn1", shots=8192)

mit = tc.results.readout_mitigation.ReadoutMit("simulator:tcn1")
mit.cals_from_system(15)

raw_count = t.results(blocked=True)
mit.apply_correction(raw_count, 15, method="M3_auto")

In [ ]:
# mitigated with m3 scalable directly on expectation: not a wrapper for count but a new algorithm!
# see eq 6 in https://arxiv.org/pdf/2006.14044.pdf

mit.expectation(raw_count, [0])

## list task and get previous task

get history tasks and their details

In [ ]:
apis.list_tasks()

In [ ]:
apis.list_tasks(device="9gmon")

In [ ]:
t = apis.get_task("d77bec2f-ab07-4dbc-a273-caa8b23a921c")

In [ ]:
t.details()

In [ ]:
t = apis.get_task("tencent::9gmon~~e32bb488-5ee9-4b07-8217-1e78ceb4bde3")

In [ ]:
t.details(prettify=True)

In [ ]:
t.results()